In [19]:
import pandas as pd
import numpy as np

In [20]:
DC_crime = pd.read_csv('Resources/one_year_dc-crimes-search-results.csv')
DC_crime.columns

Index(['NEIGHBORHOOD_CLUSTER', 'offensegroup', 'CENSUS_TRACT', 'LONGITUDE',
       'END_DATE', 'offense-text', 'YBLOCK', 'DISTRICT', 'SHIFT', 'WARD',
       'YEAR', 'offensekey', 'BID', 'sector', 'PSA', 'ucr-rank', 'BLOCK_GROUP',
       'VOTING_PRECINCT', 'XBLOCK', 'BLOCK', 'START_DATE', 'CCN', 'OFFENSE',
       'OCTO_RECORD_ID', 'ANC', 'REPORT_DAT', 'METHOD', 'location',
       'LATITUDE'],
      dtype='object')

In [21]:
DC_crime_data = pd.DataFrame(DC_crime)
# DC_crime_data.pop('NEIGHBORHOOD_CLUSTER')
# DC_crime_data.pop('CENSUS_TRACT')
# DC_crime_data.pop('LONGITUDE')
# DC_crime_data.pop('YBLOCK')
# DC_crime_data.pop('SHIFT')
# DC_crime_data.pop('DISTRICT')
# DC_crime_data.pop('WARD')
# DC_crime_data.pop('VOTING_PRECINCT')
# DC_crime_data.pop('XBLOCK')
# DC_crime_data.pop('BLOCK')
# DC_crime_data.pop('CCN')
# DC_crime_data.pop('OCTO_RECORD_ID')
# DC_crime_data.pop('ANC')
# DC_crime_data.pop('REPORT_DAT')
# DC_crime_data.pop('METHOD')
# DC_crime_data.pop('location')
# DC_crime_data.pop('LATITUDE')
# DC_crime_data.pop('END_DATE')
# DC_crime_data.pop('BID')
# DC_crime_data.pop('sector')
# DC_crime_data.pop('ucr-rank')
# DC_crime_data.pop('PSA')
# DC_crime_data.pop('BLOCK_GROUP')

DC_crime_data = DC_crime_data[['YEAR', 'START_DATE', 'offensegroup', 'OFFENSE', 'offense-text', 'offensekey']]

DC_crime_data.head()


,YEAR,START_DATE,offensegroup,OFFENSE,offense-text,offensekey
0,2022,"6/9/2017, 12:20:00 AM",violent,homicide,homicide,violent|homicide
1,2021,"2/11/2021, 3:00:52 PM",property,theft/other,theft/other,property|theft/other
2,2021,"2/11/2021, 6:03:44 PM",violent,robbery,robbery,violent|robbery
3,2021,"2/12/2021, 5:09:33 AM",violent,robbery,robbery,violent|robbery
4,2021,"2/18/2021, 4:39:52 AM",violent,robbery,robbery,violent|robbery


In [22]:
DC_crime_data[['START_DATE','START_TIME']] = DC_crime_data['START_DATE'].str.split(',',expand=True)

In [23]:
DC_crime_data.head()

,YEAR,START_DATE,offensegroup,OFFENSE,offense-text,offensekey,START_TIME
0,2022,6/9/2017,violent,homicide,homicide,violent|homicide,12:20:00 AM
1,2021,2/11/2021,property,theft/other,theft/other,property|theft/other,3:00:52 PM
2,2021,2/11/2021,violent,robbery,robbery,violent|robbery,6:03:44 PM
3,2021,2/12/2021,violent,robbery,robbery,violent|robbery,5:09:33 AM
4,2021,2/18/2021,violent,robbery,robbery,violent|robbery,4:39:52 AM


In [14]:
DC_crime_data.groupby(DC_crime_data['offensekey']).count()

,YEAR,START_DATE,offensegroup,OFFENSE,offense-text
offensekey,,,,,
property|arson,10,10,10,10,10
property|burglary,2242,2242,2242,2242,2242
property|motor vehicle theft,7562,7562,7562,7562,7562
property|theft f/auto,16476,16474,16476,16476,16476
property|theft/other,22023,22019,22023,22023,22023
violent|assault w/dangerous weapon,3038,3037,3038,3038,3038
violent|homicide,423,421,423,423,423
violent|robbery,4140,4139,4140,4140,4140
violent|sex abuse,338,338,338,338,338
